In [1]:
import numpy as np
N=100
kar = np.arange(N)*2

In [2]:
def generate_data (sig = 1e-3, A=0.1, phi_ofs=np.random.uniform(0,2*np.pi)):
    n = np.random.normal (0, 1/np.sqrt(2),N)+1j*np.random.normal(0,1/np.sqrt(2),N)
    s = A*np.exp(1j*(phi_ofs+kar*sig))
    return s+n

def estimate_phi (data,res=False,fds=False,array=False):
    sum0 = np.sum(data)
    sum1 = data*kar*1j
    sum2 = -data*kar**2    
    fd_arr = np.real(sum1)
    sd_arr = np.real(sum2)
    
    sum1 = np.sum(sum1)
    sum2 = np.sum(sum2)
    FD = np.real(sum0*np.conj(sum1))
    SD = np.real(sum2*np.conj(sum0)+sum1*np.conj(sum1))
    
    if fds:
        return FD
    elif res:
        return FD/SD
    elif array:
        return fd_arr, FD, SD
    else:
        return SD

def get_error (sig, A, phi_ofs=np.random.uniform(0,2*np.pi), Ngo=100000):
    res = [estimate_phi(generate_data(sig, A),res=True) for i in range(Ngo)]
    res = np.array(res)
    mean = res.mean()
    std = np.sqrt((res**2).mean()-sig**2)
    pred = var_phi(sig, A, Ngo)
    
    return mean, std, pred

In [3]:
def var_phi(sig, A, Ngo):
    fds = [estimate_phi(generate_data(sig, A),fds=True) for i in range(Ngo)]
    fds = np.array(fds)
    var_fd = np.var(fds)
    sd = estimate_phi(generate_data(sig, A))
    print(sd)
    print(var_fd)
    
    return var_fd/(sd**2)

In [4]:
get_error(1e-3, 5.0,0, 100000)

-839214061.2216187
41410577184.494705


(0.0010062454553975446, 0.0002744310334565159, 5.8798436758687287e-08)

In [12]:
def best_drift(Ngo,sig,A):
    drifts = []
    weights = []
    weights2 = []
    for i in range(Ngo):
        data = generate_data(sig, A)
        fd_arr, fd,sd = estimate_phi(data,array=True)
        drifts.append(fd/sd)
        weights.append(sd)
        weights2.append(np.var(fd_arr)/(sd**2))
    
    drifts = np.array(drifts)
    weights = np.array(weights)
    weights2 = np.array(weights2)
    return np.sum(drifts*weights)/np.sum(weights), np.sum(drifts*weights2)/np.sum(weights2)

In [15]:
best_drift(100000,1e-3,5.0)

(0.0010066592819741116, 0.0010016607820096122)